In [1]:
import torch
import sys
from torch import nn
import os
import random
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
import sklearn
from sklearn.metrics import roc_auc_score, log_loss, f1_score, confusion_matrix, classification_report
import timm
import cv2
from sklearn.preprocessing import LabelEncoder
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import warnings

In [ ]:
CFG = {
    'model': 'tf_efficientnet_b2_ns',
    'img_size': 260,
    'bs': 256,
    'seed': 0,
    'device': 'cuda:0',
    'img_dir': 'test_data/',
    'num_workers': 8
}


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb
    
    
class ColonDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
          
        path = "{}/{}".format(self.data_root[index], self.df.iloc[index]['image_id'])
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        return img    

def get_inference_transforms():
     return A.Compose([
            A.Resize(CFG['img_size'], CFG['img_size']),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
##### get model
class ColonImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class=2, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained, num_classes=n_class)
        try :
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, n_class)

        except :
            
            try : 
                n_features = self.model.fc.in_features
                self.model.fc = nn.Linear(n_features, n_class)
            
            except :
                try:
                    n_features = self.model.head.fc.in_channels
                    self.model.head.fc = nn.Conv2d(n_features,n_class,kernel_size=(1, 1), stride=(1, 1))
            
                except:
                    n_features = self.model.head.in_features
                    self.model.head = nn.Linear(n_features, n_class)
            
    def forward(self, x):
        x = self.model(x)
        return x

########################## inference #############################
def inference(model, data_loader, device):
    model.eval()
    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()

        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
########################## main ##################################
##########################      ##################################
test_dir = test.dir.values
warnings.filterwarnings(action='ignore') 
###### Multi GPU init
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1' #본인이 사용하고 싶은 GPU 넘버를 써주면 됨
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '53097'         # 좀 큰 숫자로 맞추면 됨 작은 숫자는 에러발생!

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# init!
torch.distributed.init_process_group(backend='nccl', init_method="env://", rank =0, world_size=1)  # rank should be 0 ~ world_size-1


if __name__ == '__main__':
    seed_everything(CFG['seed'])
    
    
    ################## get img ###########################
    tst_ds = ColonDataset(test, test_dir, transforms=get_inference_transforms())
    tst_loader = torch.utils.data.DataLoader(
        tst_ds, 
        batch_size=CFG['bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=True
    )
    
    ################## model init ########################
    device = torch.device(CFG['device'])
    model = ColonImgClassifier(CFG['model'], 2).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    model.to(device)
    model = DDP(model)
    
    
    ################## get inference #####################
    predictions = []
    model.load_state_dict(torch.load(model_dir))
    with torch.no_grad():
        predictions += [inference(model, tst_loader, device)]


    #tst_preds = inference_one_epoch(model, tst_loader, device)
    predictions = np.mean(predictions, axis=0) 
    test['pred'] = np.argmax(predictions, axis=1)
    test['confidence score'] =np.max(predictions, axis=1)
    test_acc = np.sum(test.label == test.pred) / len(test)
    test_matrix = confusion_matrix(test['label'], test['pred'])
    epoch_f1 = f1_score(test['label'], test['pred'], average='macro')


del model
torch.cuda.empty_cache()
dist.destroy_process_group()
